In [ ]:
## Code to calculate zonal statistics for OSW study sites and save as csv files
## Zonal stats separated by season for SST, chlorophyll A, sea surface height anomalies
## For master's project
## Nusrat Noor - nusratjnoor@gmail.com

In [ ]:
# Import packages
import arcpy
import geopandas as gpd
import os

In [ ]:
#Allow overwriting and set workspace
arcpy.env.overwriteOutput = True

arcpy.env.workspace = "N:/MP_Noor"


In [ ]:
# set shapefile directory
sf = '/Data/Covariates/Covariates_tool_polygon_all.shp'

sr = arcpy.Describe(sf).spatialReference

variables = ['sss','sst','sla','chlor_a']
seasons = ['fall', 'spr', 'sum', 'wint']
units = ['sss', 'sea_surface_temperature', 'sla', 'chlor_a']
    
## NetCDF file
for variable, unit in zip(variables,units):
    for season in seasons:
        fileName = f'/Data/MODIS/{variable}_{season}_months.nc'
        out_multidimensional_layer = f"{variable}_{season}_months_MD"
        arcpy.md.MakeMultidimensionalRasterLayer(
            in_multidimensional_raster=fileName,
            out_multidimensional_raster_layer=out_multidimensional_layer,
            variables=unit,
            spatial_reference=sr
        )
        output_table=f"/Data/Scratch/{variable}_{season}_stats.dbf"
        arcpy.sa.ZonalStatisticsAsTable(sf, 'NAME', out_multidimensional_layer, output_table, "DATA", "ALL", "ALL_SLICES"
                                       )


In [ ]:
# Define variables and seasons
variables = ['sss', 'sst', 'sla', 'chlor_a']
seasons = ['fall', 'spr', 'sum', 'wint']

# Define the correct directory for the .dbf files
dbf_directory = "N:/MP_Noor/Data/Scratch"

# Initialize an empty dictionary to store results
seasonal_summaries = {}

# Loop through variables and seasons
for variable in variables:
    for season in seasons:
        # Construct the DBF file name
        dbf_file = os.path.join(dbf_directory, f"{variable}_{season}_stats.dbf")
        
        # Check if the file exists before processing
        if os.path.exists(dbf_file):
            print(f"Processing {dbf_file}...")
            
            # Read the DBF file into a GeoDataFrame
            df = gpd.read_file(dbf_file)
            
            # Keep only the specified columns in your dataset
            columns_to_keep = ["NAME", "MIN", "MAX", "RANGE", "MEAN", "STD", "SUM", "MEDIAN"]
            filtered_df = df[columns_to_keep]
            
            # Group by 'NAME' and calculate summary statistics for numeric columns
            summary = filtered_df.groupby('NAME').agg(
                {col: ['mean'] for col in filtered_df.columns if col not in ['NAME']}
            )
            
            # Flatten the MultiIndex columns (optional, for clarity)
            summary.columns = ['_'.join(col).strip() for col in summary.columns]
            
            # Store the summary in the dictionary
            seasonal_summaries[f"{variable}_{season}"] = summary

            # Save the summary to a CSV file
            output_filename = f"{variable}_{season}_summary.csv"
            summary.to_csv(os.path.join(f'N:/MP_Noor/Data/Processed/', output_filename), index=True)

            print(f"Saved {output_filename}")
        else:
            print(f"{dbf_file} does not exist.")

# Example: Access the summary for 'fall' and 'sss'
fall_sss_summary = seasonal_summaries.get('sss_fall')
if fall_sss_summary is not None:
    print(fall_sss_summary.head())
else:
    print("Summary for 'sss_fall' not found.")
